In [ ]:
from dg_eval_tools import *
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from tqdm import tqdm
from model import Model

In [ ]:
model_path = "<path to model file>"
labels_path="<path to labels file>"
ground_truth_annotations_path = "<path to ground json in coco format>"
image_folder_path = "path to folder of validation images>"
# The below class map is for coco dataset. If you have custom trained model, replace with the class map of that dataset
class_map = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34,
          35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,
          64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90]  
conf_threshold=0.001
iou_threshold=0.6

In [ ]:
model = Model(model_path, labels=labels_path)
model.conf_threshold = conf_threshold
model.iou_threshold = iou_threshold
anno=COCO(ground_truth_annotations_path)
files_dict = anno.dataset["images"]
path_list = [os.path.join(image_folder_path, f["file_name"]) for f in files_dict]
img_ids = []
jdict=[]
for image_number, path in tqdm(enumerate(path_list), total=len(path_list)):
    image_id = files_dict[image_number]["id"]
    img_ids.append(image_id)
    predictions = model(path)
    save_results_coco_json(predictions.results,jdict,image_id,class_map)
pred_json = os.path.split(model_path)[-1].split('.')[0] + '_predictions.json'
with open(pred_json, 'w+') as f:
    json.dump(jdict, f)

pred = anno.loadRes(pred_json)
eval_obj = COCOeval(anno, pred, "bbox")
eval_obj.params.imgIds = sorted(img_ids) # image IDs to evaluate
eval_obj.evaluate()
eval_obj.accumulate()
eval_obj.summarize()